In [1]:
import glob
import os
import librosa
import matplotlib.pyplot as plt
import tensorflow as tf
import numpy as np
from tensorflow.contrib.keras import models
from tensorflow.contrib.keras import layers
from tensorflow.contrib import keras
%matplotlib inline
plt.style.use('ggplot')

plt.rcParams['font.family'] = 'serif'
plt.rcParams['font.serif'] = 'Ubuntu'
plt.rcParams['font.monospace'] = 'Ubuntu Mono'
plt.rcParams['font.size'] = 12
plt.rcParams['axes.labelsize'] = 11
plt.rcParams['axes.labelweight'] = 'bold'
plt.rcParams['axes.titlesize'] = 12
plt.rcParams['xtick.labelsize'] = 9
plt.rcParams['ytick.labelsize'] = 9
plt.rcParams['legend.fontsize'] = 11
plt.rcParams['figure.titlesize'] = 13

In [2]:
def windows(data, window_size):
    start = 0
    while start < len(data):
        yield int(start), int(start + window_size)
        start += (window_size / 2)

def extract_features(parent_dir,sub_dirs,file_ext="*.wav",bands = 60, frames = 41):
    window_size = 512 * (frames - 1)
    log_specgrams = []
    labels = []
    for l, sub_dir in enumerate(sub_dirs):
        for fn in glob.glob(os.path.join(parent_dir, sub_dir, file_ext)):
            sound_clip,s = librosa.load(fn)
            label = fn.split('/')[-1].split('-')[1]
            for (start,end) in windows(sound_clip,window_size):
                if(len(sound_clip[start:end]) == window_size):
                    signal = sound_clip[start:end]
                    melspec = librosa.feature.melspectrogram(signal, n_mels = bands)
                    logspec = librosa.logamplitude(melspec)
                    logspec = logspec.T.flatten()[:, np.newaxis].T
                    log_specgrams.append(logspec)
                    labels.append(label)
            
    log_specgrams = np.asarray(log_specgrams).reshape(len(log_specgrams),bands,frames,1)
    features = np.concatenate((log_specgrams, np.zeros(np.shape(log_specgrams))), axis = 3)
    for i in range(len(features)):
        features[i, :, :, 1] = librosa.feature.delta(features[i, :, :, 0])
    
    return np.array(features), np.array(labels,dtype = np.int) 

def one_hot_encode(labels):
    n_labels = len(labels)
    n_unique_labels = len(np.unique(labels))
    one_hot_encode = np.zeros((n_labels,n_unique_labels))
    one_hot_encode[np.arange(n_labels), labels] = 1
    return one_hot_encode

In [3]:
def extract_single_file_feature(fn, fn2, bands = 25, frames = 41):
    window_size = 512 * (frames - 1)
    log_specgrams = []
    labels = []
    sound_clip,s = librosa.load(fn)
    label = os.path.basename(os.path.splitext(fn)[0])
    count = 1
    for (start,end) in windows(sound_clip,window_size):
        if(len(sound_clip[start:end]) == window_size):
            print('\nwindow %d shapes:'%count)
            signal = sound_clip[start:end]
            melspec = librosa.feature.melspectrogram(signal, n_mels = bands)
            print('melspec.shape - %s'%str(melspec.shape))
            logspec = librosa.logamplitude(melspec)
            print('logspec.shape - %s'%str(logspec.shape))
            logspec = logspec.T.flatten()[:, np.newaxis].T
            print('logspec.shape - %s'%str(logspec.shape))
            log_specgrams.append(logspec)
            labels.append(label)
            count = count + 1
            
    sound_clip,s = librosa.load(fn2)
    label = os.path.basename(os.path.splitext(fn2)[0])
    count = 1
    for (start,end) in windows(sound_clip,window_size):
        if(len(sound_clip[start:end]) == window_size):
            print('\nwindow %d shapes:'%count)
            signal = sound_clip[start:end]
            melspec = librosa.feature.melspectrogram(signal, n_mels = bands)
            print('melspec.shape - %s'%str(melspec.shape))
            logspec = librosa.logamplitude(melspec)
            print('logspec.shape - %s'%str(logspec.shape))
            logspec = logspec.T.flatten()[:, np.newaxis].T
            print('logspec.shape - %s'%str(logspec.shape))
            log_specgrams.append(logspec)
            labels.append(label)
            count = count + 1


    log_specgrams = np.asarray(log_specgrams).reshape(len(log_specgrams),bands,frames,1)
    print('\nlog_specgrams.shape - %s'%str(log_specgrams.shape))
    features = np.concatenate((log_specgrams, np.zeros(np.shape(log_specgrams))), axis = 3)
    print('features.shape - %s'%str(features.shape))
    print('len of features = %s'%str(len(features)))
    for i in range(len(features)):
        features[i, :, :, 1] = librosa.feature.delta(features[i, :, :, 0])
    print('features.shape - %s'%str(features.shape))
    return np.array(features), np.array(labels,dtype = np.int) 

In [36]:
ft, lb = extract_single_file_feature('/Users/Gundeep/Dropbox/notebook/sounds/recorded/large/ah_drakht_boht_sohna_lag_reha/1.wav','/Users/Gundeep/Dropbox/notebook/sounds/recorded/large/ah_drakht_boht_sohna_lag_reha/3.wav')


window 1 shapes:
melspec.shape - (25, 41)
logspec.shape - (25, 41)
logspec.shape - (1, 1025)

window 2 shapes:
melspec.shape - (25, 41)
logspec.shape - (25, 41)
logspec.shape - (1, 1025)

window 3 shapes:
melspec.shape - (25, 41)
logspec.shape - (25, 41)
logspec.shape - (1, 1025)

window 4 shapes:
melspec.shape - (25, 41)
logspec.shape - (25, 41)
logspec.shape - (1, 1025)

window 5 shapes:
melspec.shape - (25, 41)
logspec.shape - (25, 41)
logspec.shape - (1, 1025)

window 1 shapes:
melspec.shape - (25, 41)
logspec.shape - (25, 41)
logspec.shape - (1, 1025)

window 2 shapes:
melspec.shape - (25, 41)
logspec.shape - (25, 41)
logspec.shape - (1, 1025)

window 3 shapes:
melspec.shape - (25, 41)
logspec.shape - (25, 41)
logspec.shape - (1, 1025)

window 4 shapes:
melspec.shape - (25, 41)
logspec.shape - (25, 41)
logspec.shape - (1, 1025)

window 5 shapes:
melspec.shape - (25, 41)
logspec.shape - (25, 41)
logspec.shape - (1, 1025)

log_specgrams.shape - (10, 25, 41, 1)
features.shape - (10,

In [37]:
lb

array([1, 1, 1, 1, 1, 3, 3, 3, 3, 3])

In [6]:
parent_dir = '/home/paperspace/Documents/Projects/Urban Sound Classification/UrbanSound8K/audio/'
sub_dirs= ['fold1','fold2']
features,labels = extract_features(parent_dir,sub_dirs)
labels = one_hot_encode(labels)

### Dump to pickle

In [7]:
rnd_indices = np.random.rand(len(labels)) < 0.70

train_x = features[rnd_indices]
train_y = labels[rnd_indices]
test_x = features[~rnd_indices]
test_y = labels[~rnd_indices]

In [9]:
# dump to pickle
if os.environ['HOME'] == '/Users/Gundeep':
    pickle_filename = 'urban_pickle_cnn.pickle'
else:
    pickle_filename = '/home/paperspace/Documents/Projects/Urban Sound Classification/urban_pickle_cnn.pickle'

pickle_dump = {'train_x' : train_x,
               'train_y' : train_y,
               'test_x' : test_x,
               'test_y' : test_y}
import pickle
try:
  with open(pickle_filename, 'wb') as f:
    pickle.dump(pickle_dump, f, pickle.HIGHEST_PROTOCOL)
except Exception as e:
  print('Unable to save data to', pickle_filename, ':', e)

### Load from pickle

In [3]:
# load from pickle
import pickle
if os.environ['HOME'] == '/Users/Gundeep':
    pickle_filename = 'urban_pickle_cnn.pickle'
else:
    pickle_filename = '/home/paperspace/Documents/Projects/Urban Sound Classification/urban_pickle_cnn.pickle'
pickle_dump = {}
try:
  with open(pickle_filename, 'rb') as f:
    pickle_dump = pickle.load(f)
except Exception as e:
  print('Unable to load pickle', pickle_filename, ':', e)

In [5]:
train_x = pickle_dump['train_x']
train_y = pickle_dump['train_y']
test_x = pickle_dump['test_x']
test_y = pickle_dump['test_y']

In [6]:
print('train_x:%s'%str(train_x.shape))
print('train_y%s'%str(train_y.shape))
print('test_x%s'%str(test_x.shape))
print('test_y:%s'%str(test_y.shape))

train_x:(7594, 60, 41, 2)
train_y(7594, 10)
test_x(3240, 60, 41, 2)
test_y:(3240, 10)


### Create neural net

In [7]:
# keras class shorthands
Sequential = models.Sequential
Dense = layers.Dense
Conv2D = layers.Conv2D

In [8]:
frames = 41
bands = 60

feature_size = 2460 #60x41
num_classes = 10
num_channels = 2
epochs = 2000

batch_size = 50
kernel_size = 30
depth = 20
num_hidden = 200

learning_rate = 0.01
training_iterations = 2000

In [9]:
train_x.shape[1:]

(60, 41, 2)

In [10]:
model = Sequential()
model.add(Conv2D(depth, (5,5), input_shape=train_x.shape[1:], activation='relu', padding='same'))
model.add(layers.Flatten())
model.add(Dense(num_hidden, activation='sigmoid'))
model.add(Dense(num_classes, activation='softmax'))

In [11]:
opt = keras.optimizers.Adam(lr=learning_rate)#, decay=1e-6)

In [12]:
model.compile(loss='categorical_crossentropy', optimizer=opt, metrics=['accuracy'])

### setup file logger

In [14]:
from keras.callbacks import CSVLogger
csv_logger = CSVLogger('/home/paperspace/Documents/log.csv', append=True, separator=';')

In [ ]:
model.fit(train_x, train_y,
              batch_size=batch_size,
              epochs=epochs,
              validation_data=(test_x, test_y),
              shuffle=True,
              callbacks=[csv_logger])

Train on 7594 samples, validate on 3240 samples
Epoch 1/2000
7594/7594 [==============================] - 5s - loss: 2.2292 - acc: 0.1625 - val_loss: 2.2812 - val_acc: 0.1654
Epoch 2/2000
7594/7594 [==============================] - 3s - loss: 2.1836 - acc: 0.1836 - val_loss: 2.1100 - val_acc: 0.1960
Epoch 3/2000
7594/7594 [==============================] - 3s - loss: 2.1898 - acc: 0.1603 - val_loss: 2.2531 - val_acc: 0.1642
Epoch 4/2000
7594/7594 [==============================] - 3s - loss: 2.1942 - acc: 0.1628 - val_loss: 2.1620 - val_acc: 0.1667
Epoch 5/2000
7594/7594 [==============================] - 3s - loss: 2.1584 - acc: 0.1707 - val_loss: 2.1555 - val_acc: 0.1741
Epoch 6/2000
7594/7594 [==============================] - 3s - loss: 2.1404 - acc: 0.1809 - val_loss: 2.3227 - val_acc: 0.2022
Epoch 7/2000
7594/7594 [==============================] - 3s - loss: 2.1499 - acc: 0.1882 - val_loss: 2.0639 - val_acc: 0.2191
Epoch 8/2000
7594/7594 [==============================] - 3s - 

7594/7594 [==============================] - 3s - loss: 2.1714 - acc: 0.1749 - val_loss: 2.1553 - val_acc: 0.1861
Epoch 65/2000
7594/7594 [==============================] - 3s - loss: 2.1612 - acc: 0.1716 - val_loss: 2.1526 - val_acc: 0.1556
Epoch 66/2000
7594/7594 [==============================] - 3s - loss: 2.1637 - acc: 0.1676 - val_loss: 2.1462 - val_acc: 0.17310.1
Epoch 67/2000
7594/7594 [==============================] - 3s - loss: 2.1568 - acc: 0.1653 - val_loss: 2.1636 - val_acc: 0.1738
Epoch 68/2000
7594/7594 [==============================] - 3s - loss: 2.1475 - acc: 0.1741 - val_loss: 2.2242 - val_acc: 0.1833
Epoch 69/2000
7594/7594 [==============================] - 3s - loss: 2.1615 - acc: 0.1662 - val_loss: 2.1816 - val_acc: 0.1867
Epoch 70/2000
7594/7594 [==============================] - 3s - loss: 2.1818 - acc: 0.1692 - val_loss: 2.2366 - val_acc: 0.1914
Epoch 71/2000
7594/7594 [==============================] - 3s - loss: 2.1390 - acc: 0.1790 - val_loss: 2.1013 - val

7594/7594 [==============================] - 3s - loss: 2.1464 - acc: 0.1736 - val_loss: 2.2062 - val_acc: 0.1580
Epoch 128/2000
7594/7594 [==============================] - 3s - loss: 2.1416 - acc: 0.1799 - val_loss: 2.2363 - val_acc: 0.1562
Epoch 129/2000
7594/7594 [==============================] - 3s - loss: 2.1359 - acc: 0.1712 - val_loss: 2.1962 - val_acc: 0.1472
Epoch 130/2000
7594/7594 [==============================] - 3s - loss: 2.1407 - acc: 0.1655 - val_loss: 2.1759 - val_acc: 0.1917
Epoch 131/2000
7594/7594 [==============================] - 3s - loss: 2.1646 - acc: 0.1701 - val_loss: 2.1699 - val_acc: 0.1503
Epoch 132/2000
7594/7594 [==============================] - 3s - loss: 2.1579 - acc: 0.1682 - val_loss: 2.1358 - val_acc: 0.1593
Epoch 133/2000
7594/7594 [==============================] - 3s - loss: 2.1768 - acc: 0.1643 - val_loss: 2.2322 - val_acc: 0.1472
Epoch 134/2000
7594/7594 [==============================] - 3s - loss: 2.1503 - acc: 0.1767 - val_loss: 2.1322 -

7594/7594 [==============================] - 3s - loss: 2.1457 - acc: 0.1712 - val_loss: 2.1264 - val_acc: 0.1750
Epoch 191/2000
7594/7594 [==============================] - 3s - loss: 2.1564 - acc: 0.1726 - val_loss: 2.1178 - val_acc: 0.1935
Epoch 192/2000
7594/7594 [==============================] - 3s - loss: 2.1352 - acc: 0.1755 - val_loss: 2.1272 - val_acc: 0.1877
Epoch 193/2000
7594/7594 [==============================] - 3s - loss: 2.1474 - acc: 0.1704 - val_loss: 2.1694 - val_acc: 0.1475
Epoch 194/2000
7594/7594 [==============================] - 3s - loss: 2.1616 - acc: 0.1774 - val_loss: 2.1286 - val_acc: 0.1722
Epoch 195/2000
7594/7594 [==============================] - 3s - loss: 2.1516 - acc: 0.1705 - val_loss: 2.1437 - val_acc: 0.1654
Epoch 196/2000
7594/7594 [==============================] - 3s - loss: 2.1376 - acc: 0.1804 - val_loss: 2.1247 - val_acc: 0.1747
Epoch 197/2000
7594/7594 [==============================] - 3s - loss: 2.1453 - acc: 0.1763 - val_loss: 2.1442 -

7594/7594 [==============================] - 3s - loss: 2.1334 - acc: 0.1708 - val_loss: 2.2344 - val_acc: 0.1870
Epoch 254/2000
7594/7594 [==============================] - 3s - loss: 2.1500 - acc: 0.1779 - val_loss: 2.1820 - val_acc: 0.1873
Epoch 255/2000
7594/7594 [==============================] - 3s - loss: 2.1573 - acc: 0.1701 - val_loss: 2.1459 - val_acc: 0.18890.170
Epoch 256/2000
7594/7594 [==============================] - 3s - loss: 2.1491 - acc: 0.1759 - val_loss: 2.1502 - val_acc: 0.1889
Epoch 257/2000
7594/7594 [==============================] - 3s - loss: 2.1578 - acc: 0.1678 - val_loss: 2.2077 - val_acc: 0.1577
Epoch 258/2000
7594/7594 [==============================] - 3s - loss: 2.1613 - acc: 0.1625 - val_loss: 2.1303 - val_acc: 0.1926
Epoch 259/2000
7594/7594 [==============================] - 3s - loss: 2.1535 - acc: 0.1787 - val_loss: 2.1972 - val_acc: 0.1571
Epoch 260/2000
7594/7594 [==============================] - 3s - loss: 2.1663 - acc: 0.1753 - val_loss: 2.1

7594/7594 [==============================] - 3s - loss: 2.1634 - acc: 0.1763 - val_loss: 2.1506 - val_acc: 0.1660
Epoch 317/2000
7594/7594 [==============================] - 3s - loss: 2.1491 - acc: 0.1742 - val_loss: 2.1658 - val_acc: 0.1929
Epoch 318/2000
7594/7594 [==============================] - 3s - loss: 2.1483 - acc: 0.1708 - val_loss: 2.1384 - val_acc: 0.1361
Epoch 319/2000
7594/7594 [==============================] - 3s - loss: 2.1545 - acc: 0.1716 - val_loss: 2.1572 - val_acc: 0.1877
Epoch 320/2000
7594/7594 [==============================] - 3s - loss: 2.1488 - acc: 0.1738 - val_loss: 2.1590 - val_acc: 0.1568
Epoch 321/2000
7594/7594 [==============================] - 3s - loss: 2.1345 - acc: 0.1663 - val_loss: 2.1395 - val_acc: 0.1657
Epoch 322/2000
7594/7594 [==============================] - 3s - loss: 2.1256 - acc: 0.1754 - val_loss: 2.1468 - val_acc: 0.1883
Epoch 323/2000
7594/7594 [==============================] - 3s - loss: 2.1457 - acc: 0.1724 - val_loss: 2.1389 -